# Eval

In [ ]:
#| default_exp eval

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2
# %load_ext rich

In [ ]:
#| export
import os
import json
import yaml
from stringdale import (
    Define,
    Scope,
    V,
    E,
    Condition,
    draw_nx
)
from stringdale.stream_warping import (
    TestCase,
    parse_test_case,
    TraceLog,
    event_stream_warp,
    word_overlap,
    regex,
)


from pathlib import Path
from frozendict import frozendict
from stringdale.core import  checkLogs
import pytest
import asyncio
from pydantic import BaseModel, ConfigDict

from typing import List, Union
import jsonlines
import logging


In [ ]:
#| export
logger = logging.getLogger(__name__)

## Using podtw

In [ ]:
# TODO multiple inputs
# TODO ANY comparison so we can check for existance of nodes

In [ ]:
#| export

def parse_trace_log(trace_path:Union[str,Path]) -> TraceLog:
    """
    Parse a trace file into a list of Trace objects.
    """
    with jsonlines.open(trace_path) as reader:
        traces = [trace for trace in reader]
        return TraceLog(steps=traces)

In [ ]:
from stringdale.core import get_git_root

In [ ]:
sample_data_dir = get_git_root() / "sample_data" / "eval"

example_trace_log_path = sample_data_dir / "traces0.jsonl"
example_case_path = sample_data_dir / "expected0.yaml"


example_comparisons = {
    "word_overlap":word_overlap,
    "regex":regex,
}

In [ ]:
example_test_case = parse_test_case(example_case_path)
example_trace_log  = parse_trace_log(example_trace_log_path)

In [ ]:
best_match,score,debug_info =await event_stream_warp(example_trace_log,example_test_case,comparisons=example_comparisons,default_comparison=word_overlap)
best_match

frozendict.frozendict({'3': 2, 'node_a1': 1, 'node_z': 4, '1': 5})

In [ ]:
# debug_info

## Realistic Comparison Functions

In [ ]:
#| export
import numpy as np
import asyncio
from stringdale.db import openai_embed
from stringdale.chat import Chat

In [ ]:
#| export
async def cosine_dist(out: str, expected: str, model: str = 'text-embedding-3-small') -> float:
    """Compute cosine distance between two strings using OpenAI embeddings.
    
    Args:
        out: First string to compare
        expected: Second string to compare
        model: OpenAI embedding model to use (default: 'text-embedding-3-small')
        
    Returns:
        float: Cosine similarity between the two strings (between -1 and 1)
    """
    # Get embeddings for both strings
    if not isinstance(out,str):
        return np.inf
    if not isinstance(expected,str):
        raise ValueError(f"cosine_dist: expected is not a string: {expected}")
    out_embedding = await openai_embed(out, model=model)
    expected_embedding = await openai_embed(expected, model=model)
    
    # Compute cosine similarity
    dot_product = np.dot(out_embedding, expected_embedding)
    norm_out = np.linalg.norm(out_embedding)
    norm_expected = np.linalg.norm(expected_embedding)
    
    # Return cosine similarity
    return 1-dot_product / (norm_out * norm_expected)



In [ ]:
basic_dist = await cosine_dist("hello","hello")
basic_dist
assert basic_dist < 0.0001


In [ ]:
await cosine_dist("hello","hello stranger")

np.float64(0.3944818489490096)

In [ ]:
#|export
from stringdale.core import jinja_undeclared_vars
from typing import Any

In [ ]:
class ChatEvalScore(BaseModel):
    score:float


async def chat_eval(out:Any,expected:Any,model:str="gpt-4o-mini",system_prompt:str=None)->float:

    if system_prompt is None:
        system_prompt = """
            You are a helpful assistant that evaluates the similarity of two strings.
            You will be given two strings, and you will need to evaluate the similarity of the two strings.
            You will need to return a score between 0 and 1, where 0 is the lowest similarity and 1 is the highest similarity.

            string1: {{out}}
            string2: {{expected}}

            return a score between 0 and 1, where 0 is the lowest similarity and 1 is the highest similarity.
            """

    if not jinja_undeclared_vars(system_prompt) == {'out','expected'}:
        raise ValueError("System prompt must contain {{out}} and {{expected}} jinja variables")

    chat = Chat(model=model,messages=
        [{"role":"system","content":system_prompt}],
        output_schema=ChatEvalScore,
        out = out,
        expected = expected,
        )
    response = await chat()
    return response['content'].score

In [ ]:
custom_prompt = """
if one of the strings contains "hello", return 0.5

string1: {{out}}
string2: {{expected}}
"""


result = await chat_eval("hello","world",system_prompt=custom_prompt)
assert result == 0.5

In [ ]:
custom_prompt = """
if one of the strings contains "hello", return 0.5

string1: {{outs}}
string2: {{expected}}
"""

with pytest.raises(ValueError,match="System prompt must contain {{out}} and {{expected}} jinja variables"):
    result = await chat_eval("hello","world",system_prompt=custom_prompt)


In [ ]:
#| export
def eq(a,b):
    if a == b:
        return 0
    else:
        return np.inf

def any(a,b):
    return 0

In [ ]:
out = 3500
expr = "({0} < 4000) & ({0} > 3000)"

In [ ]:
f_expr = expr.format(out)
f_expr

'(3500 < 4000) & (3500 > 3000)'

In [ ]:
#| export
from stringdale.tools import run_python_code


In [ ]:
#| export
def safe_eval(out,expression):
    try:
        formatted_expressions = expression.format(out)
    except Exception as e:
        logger.warning(f"Error formatting expression: {expression} with value {out}, error: {e}")
        return np.inf
    value = run_python_code(formatted_expressions)
    if isinstance(value,str) and value.startswith("Error"):
        logger.warning(
            f"Error evaluating expression: {formatted_expressions} = {value}\n"
            f"out: {out}\n"
            f"expression: {expression}\n"
            f"error: {e}"
        )
        return np.inf
    logger.debug(f"safe_eval: {formatted_expressions} = {value}")
    if isinstance(value,bool):
        return 0 if value else np.inf
    elif isinstance(value,float):
        return value
    else:
        logger.debug(
            f"When evaluating {expression} with value {out}\n"
            f"Expected float or bool, got {type(value)} with value {repr(value)}"
            )
        return np.inf

In [ ]:
eval_string ="""
x=4000
({0} < x) & ({0} > 3000)
"""

with checkLogs():
    y =safe_eval(3500,eval_string)
y

__main__ - DEBUG - safe_eval: 
x=4000
(3500 < x) & (3500 > 3000)
 = True


0

In [ ]:
safe_eval(3500,"""
x=4000
({0} < x) & ({0} > 3000)
""")

0

## Running and evaluating a single data point

In [ ]:
#| export
from typing import List,Dict,Callable

In [ ]:
#| export
class DataPoint(BaseModel):
    traces:TraceLog
    expected:TestCase
    

In [ ]:
#| export

async def _run_agent(Agent,test_case:TestCase,trace_log_path:Path):
    d=Agent()
    with jsonlines.open(trace_log_path,'w') as writer:
        for input in test_case.inputs:
            async for trace in d.arun(input):
                writer.write(json.loads(trace.model_dump_json(include={'name','output','duration'})))
            if d.finished:
                break

async def evaluate_datapoint(Agent,comparisons,default_comparison,test_case_path,trace_log_path=None,force_run=False):
    if trace_log_path is None:
        trace_log_path = test_case_path.parent/test_case_path.name.replace(".yaml", ".jsonl").replace("expected", "actual")

    if not trace_log_path.parent.exists():
        os.makedirs(trace_log_path.parent,exist_ok=True)
    try:
        test_case = parse_test_case(test_case_path)
    except Exception as e:
        raise ValueError(f"Error parsing test case {test_case_path}: {e}") from e
        

    if force_run or not trace_log_path.exists():
        if not trace_log_path.exists():
            logger.info(f"Trace file {trace_log_path.name} does not exist, running agent")
        else:
            logger.info(f"Force running {trace_log_path.name}")
        await _run_agent(Agent,test_case,trace_log_path)
    else:
        logger.info(f"Trace file {trace_log_path.name} already exists, skipping agent run")

    parsed_trace = parse_trace_log(trace_log_path)
    aligned_trace,score,debug_info = await event_stream_warp(parsed_trace,test_case,comparisons,default_comparison)
    
    return aligned_trace,score,debug_info,trace_log_path


In [ ]:
from stringdale.examples.react import ReactAgent

In [ ]:
agent = ReactAgent
expected_yaml = sample_data_dir/"react_expected.yaml"
bad_expected_yaml = sample_data_dir/"react_bad_expected.yaml"
comparisons = {
    "eq":eq,
    "eval":safe_eval,
}
default_comparison = cosine_dist


In [ ]:
#| export

with checkLogs():
    alignment,score,debug_info,trace_out = await evaluate_datapoint(agent,comparisons,default_comparison,bad_expected_yaml)

assert alignment is None
alignment,score,trace_out


__main__ - INFO - Trace file react_bad_actual.jsonl already exists, skipping agent run
No viable trace row nums for expected trace 1
No possible mappings found


(None,
 inf,
 PosixPath('/Users/dean/dl/stringdale/sample_data/eval/react_bad_actual.jsonl'))

In [ ]:
with checkLogs(level='INFO'):
    alignment,score,debug_info,trace_out = await evaluate_datapoint(agent,comparisons,default_comparison,expected_yaml)

assert dict(alignment) == {'0': 2, '1': 8}
alignment,score,trace_out

__main__ - INFO - Trace file react_actual.jsonl already exists, skipping agent run


(frozendict.frozendict({'0': 2, '1': 8}),
 np.float64(0.3244313858854829),
 PosixPath('/Users/dean/dl/stringdale/sample_data/eval/react_actual.jsonl'))

In [ ]:
#| export
import pandas as pd

In [ ]:
#| export

def _pd_order_columns_first(df:pd.DataFrame,first_columns:list[str]):
    """
    Reorder the columns of a pandas dataframe to put the first_columns first.
    """
    return df[first_columns + [c for c in df.columns if c not in first_columns]]



In [ ]:
x = pd.DataFrame([
    {'distance':1,'comparison':'eq','actual':1,'expected':1},
    {'distance':2,'comparison':'eq','actual':2,'expected':2},
    {'distance':3,'comparison':'eq','actual':3,'expected':3},
])

_pd_order_columns_first(x,['actual','expected'])

,actual,expected,distance,comparison
0,1,1,1,eq
1,2,2,2,eq
2,3,3,3,eq


In [ ]:
# debug_info

In [ ]:
#| export
from copy import deepcopy
from itertools import count

In [ ]:
#| export

def summarize_datapoint(name,alignment,debug_info):
    """
    Summarize the datapoint by getting the distance per step and total metrics such as sum of distances and coverage
    by using the alignment and the debug info
    """
    deep_dive_fit = []

    comp_counter = count()
    for expected_node_id,trace_idx in alignment.items():
        match_data = debug_info[expected_node_id][trace_idx]
        for comp in match_data['comparisons']:
            summary = deepcopy(match_data) | deepcopy(comp) 
            summary['comp_id'] = next(comp_counter)
            summary.pop('comparisons')
            deep_dive_fit.append(summary)

    df = pd.DataFrame(deep_dive_fit)
    df['datapoint'] = str(name)
    df = _pd_order_columns_first(df,['datapoint','node_label','trace_idx','comparison','key','actual','expected','distance'])
    return df

In [ ]:
df = summarize_datapoint('react',alignment,debug_info)
df

,datapoint,node_label,trace_idx,comparison,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,comp_id
0,react,0,2,eq,content.name,wikipedia_search,wikipedia_search,0.000000,0,use_tool,use_tool,{},0
1,react,0,2,None,content.input.q,Barack Obama,Obama,0.324431,0,use_tool,use_tool,{},1
2,react,1,8,eq,content.name,run_python_code,run_python_code,0.000000,1,use_tool,use_tool,{},2
3,react,1,8,eval,content.output,3844,({0} < 4000) & ({0} > 3000),0.000000,1,use_tool,use_tool,{},3


In [ ]:
assert df['node_label'].to_list() == ['0','0','1','1']
assert df['key'].to_list() == ['content.name','content.input.q','content.name','content.output']

## DataSets

In [ ]:
#| export
from stringdale import DiagramSchema
from pprint import pprint, pformat

In [ ]:
#| export
def _trace_out_path(expected_yaml:Path,expected_dir:Path,trace_dir:Path):
    return trace_dir / expected_yaml.relative_to(expected_dir).with_suffix(".jsonl")

def _find_yamls(test_dir:Path):
    expected_yamls = list(test_dir.glob("**/*.yaml")) + list(test_dir.glob("**/*.yml"))
    return expected_yamls


def trace_log_len(trace_log_path:Path):
    """Gets number of traces in a trace log file
    This is equal to the number of nodes executed in the workflow
    """
    return len(trace_log_path.read_text().splitlines())

def serialize_test_case(test_case:Path):
    yml_version = yaml.safe_load(test_case.read_text())
    json_version = json.dumps(yml_version,indent=2)
    return json_version

In [ ]:
#| export
class TestSetRun(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    test_dir: Path
    dir: Path
    summary: pd.DataFrame
    details: pd.DataFrame
    debug: dict

    def __repr__(self):     
        return (
            f"TestSetRun(\n"
            f"  test_dir={self.test_dir}, \n"
            f"  dir={self.log_dir}, \n"
            f"  summary=Dataframe({self.summary.shape}), \n"
            f"  details=Dataframe({self.details.shape}), \n"
            f"  debug=dict)")

    def __str__(self):
        return self.__repr__()

    def save(self,dir:Path):
        self.summary.to_csv(dir/"summary.csv")
        self.details.to_csv(dir/"details.csv")
        (dir/'test_cases_loc.txt').write_text(str(self.test_dir.relative_to(self.dir)))
        with open(dir/"debug.json","w") as f:
            json.dump(self.debug,f)

    @classmethod
    def load(cls, dir: Path):
        # Initialize empty DataFrames and dict for missing files
        summary = pd.DataFrame(columns=['datapoint','distance','avg_distance','coverage','alignment','serialized_test_case'])
        details = pd.DataFrame()
        debug = {}
        test_cases_loc = dir  # Default to the input directory
        
        # Try to load files if they exist
        try:
            if (dir/"summary.csv").exists():
                summary = pd.read_csv(dir/"summary.csv")
            if (dir/"details.csv").exists():
                details = pd.read_csv(dir/"details.csv")
            if (dir/"debug.json").exists():
                with open(dir/"debug.json") as f:
                    debug = json.load(f)
            if (dir/'test_cases_loc.txt').exists():
                test_cases_loc = (dir/'test_cases_loc.txt').read_text().strip()
                test_cases_loc = dir/test_cases_loc
        except Exception as e:
            # Log the error but continue with empty/default values
            print(f"Warning: Error loading some files: {str(e)}")
        
        return cls(
            test_dir=test_cases_loc,
            dir=dir,
            summary=summary,
            details=details,
            debug=debug
        )


    def trace_log_path(self,datapoint:str):
        return self.dir/'logs'/datapoint/'.jsonl'
# TODO from here, factor all TestSetRun file logic to this class
    def datapoint_len(self,datapoint:str):
        datapoint_yaml = (self.test_dir/datapoint).with_suffix(".yaml")
        return trace_log_len(self.dir/datapoint_yaml)

    def serialize_test_case(self,datapoint_path):
        pass

    def is_datapoint_stale(self,datapoint_path):
        pass
        # if the datapoint is not in the summary, it is stale
        # load_serialized_test_case
        # if it is in summary but the serialized_test_case is different, it is stale
        # else, not stale


In [ ]:
#| export

async def eval_dataset(Agent:DiagramSchema,test_dir,out_dir,comparisons,default_comparison,force_run=False):

    # TODO from here
    # check if log_dir has a summary (only if not force_run)
    # if so, load it, and compare the test_case_json to the existing_yaml
    # only if it is different, and the datapoint to the datapoints to be run
    # since we computed ourselves which datapoints to run, we call evaluate_datapoint with force_run=True

    run = TestSetRun.load(out_dir)

    # TODO
    # get_all_yamls
    # filter to stale ones
    # only run the stale ones


    test_cases = _find_yamls(test_dir)
    relative_test_cases = [test_cases.relative_to(test_dir) for test_cases in test_cases]

    trace_files  = [_trace_out_path(test_case,test_dir,out_dir/'logs') for test_case in test_cases]

    logger.info(f"Evaluating {len(test_cases)} datapoints, logging to {out_dir/'logs'}")
    datapoint_tasks = [evaluate_datapoint(
            Agent=Agent,
            comparisons=comparisons,
            default_comparison=default_comparison,
            test_case_path=test_case,
            trace_log_path=trace_file,
            force_run=force_run,
        ) for test_case,trace_file in zip(test_cases,trace_files) if trace_file in trace_files]
    
    datapoint_results = await asyncio.gather(*datapoint_tasks,return_exceptions=True)

    for result,relative_test_case in zip(datapoint_results,relative_test_cases):
      if isinstance(result,Exception):
        result.args = (f"When evaluating datapoint {relative_test_case}:\n{result.args[0]}", )+ result.args[1:]
        raise result

    summary_data = list()
    deep_dives = list()
    debug_infos = dict()

    for (test_case,alignment,score,debug_info,trace_out),test_case_path in zip(datapoint_results,test_cases):
        datapoint_name = trace_out.relative_to(log_dir).with_suffix("")
        serialized_test_case = serialize_test_case(test_case_path)
        deep_dive = summarize_datapoint(datapoint_name,alignment,debug_info)
        deep_dives.append(deep_dive)
        avg_distance = deep_dive.distance.mean()
        coverage = len(alignment) / trace_log_len(trace_out)
        summary = {'datapoint':str(datapoint_name),'distance':score,'avg_distance':avg_distance,'coverage':coverage,'alignment':alignment,'serialized_test_case':serialized_test_case}
        summary_data.append(summary)
        debug_infos[datapoint_name] = debug_info
    

    # update the existing summary and details dataframes
    summary_df = pd.DataFrame(summary_data)
    if len(deep_dives) > 0:
        deep_dives_df = pd.concat(deep_dives).reset_index(drop=True)
    else:
        deep_dives_df = pd.DataFrame()

    return TestSetRun(
        test_dir=test_dir,
        log_dir=log_dir,
        summary=summary_df,
        details=deep_dives_df,
        debug=debug_infos
    )

In [ ]:
from stringdale.examples.rag import Rag
from stringdale.db import ChromaClient

In [ ]:
def load_agent(conf_dir: Path):
    agent_yaml_path = conf_dir / 'agent.yml'
    vec_db_yaml_path = conf_dir / 'vec_db.yml'

    agent_conf = yaml.safe_load(agent_yaml_path.read_text())
    vec_db_conf = yaml.safe_load(vec_db_yaml_path.read_text())

    db = ChromaClient()
    for collection_name, docs in vec_db_conf.items():
        db.add_collection(collection_name, exists_ok=True)
        db.upsert(collection_name, docs)

    agent_conf['db'] = db
    
    Agent = Rag(**agent_conf)

    return Agent

In [ ]:
test_dir = get_git_root() / "sample_data" / "eval_datasets" / "test_cases"
log_dir = get_git_root() / "sample_data" / "eval_datasets" / "trace_logs"
conf_dir = get_git_root() / "sample_data" / "eval_datasets" / "agent_configs"


comparisons = {
    'eq':eq,
    'eval':safe_eval,
    'chat_eval':chat_eval,
    'cosine_dist':cosine_dist,
    # TODO make a chat_eval where you can put the system prompt as a kwarg
}

default_comparison = cosine_dist

In [ ]:
with checkLogs(level='INFO'):
    run1 = await eval_dataset(
        Agent=load_agent(conf_dir/'v001'),
        test_dir=test_dir,
        log_dir=log_dir/'v001',
        comparisons=comparisons,
        default_comparison=default_comparison,
        )

    run2 = await eval_dataset(
        Agent=load_agent(conf_dir/'v002'),
        test_dir=test_dir,
        log_dir=log_dir/'v002',
        comparisons=comparisons,
        default_comparison=default_comparison)

    run3 = await eval_dataset(
        Agent=load_agent(conf_dir/'v003'),
        test_dir=test_dir,
        log_dir=log_dir/'v003',
        comparisons=comparisons,
        default_comparison=default_comparison)



__main__ - INFO - Evaluating 2 datapoints, logging to /Users/dean/dl/stringdale/sample_data/eval_datasets/trace_logs/v001
__main__ - INFO - Trace file pikachus.jsonl already exists, skipping agent run
__main__ - INFO - Trace file goldens.jsonl already exists, skipping agent run
__main__ - INFO - Evaluating 2 datapoints, logging to /Users/dean/dl/stringdale/sample_data/eval_datasets/trace_logs/v002
__main__ - INFO - Trace file pikachus.jsonl already exists, skipping agent run
__main__ - INFO - Trace file goldens.jsonl already exists, skipping agent run
__main__ - INFO - Evaluating 2 datapoints, logging to /Users/dean/dl/stringdale/sample_data/eval_datasets/trace_logs/v003
__main__ - INFO - Trace file pikachus.jsonl already exists, skipping agent run
__main__ - INFO - Trace file goldens.jsonl already exists, skipping agent run


In [ ]:
run1

TestSetRun(
  test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
  log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/trace_logs/v001, 
  summary=Dataframe((2, 5)), 
  details=Dataframe((3, 13)), 
  debug=dict)

In [ ]:
run1.summary

,datapoint,distance,avg_distance,coverage,alignment
0,pikachus,0.943595,0.943595,0.25,{'0': 2}
1,goldens,0.751374,0.375687,0.50,"{'0': 1, '1': 2}"


In [ ]:
run1.details

,datapoint,node_label,trace_idx,comparison,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,comp_id
0,pikachus,0,2,None,content,"I'm sorry, but I can only provide information ...",They are:\n * dangerous\n * smoke cigarettes,0.943595,0,chat,chat,{},0
1,goldens,0,1,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,0
2,goldens,1,2,None,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.651374,1,chat,chat,{},1


In [ ]:
run2.details

,datapoint,node_label,trace_idx,comparison,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,comp_id
0,pikachus,0,2,None,content,"Pikachus are not dogs, they are fictional crea...",They are:\n * dangerous\n * smoke cigarettes,0.818815,0,chat,chat,{},0
1,goldens,0,1,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,0
2,goldens,1,2,None,content,"Golden Retrievers are friendly, intelligent do...",They are:\n * friendly\n * intelligent\n,0.602882,1,chat,chat,{},1


In [ ]:
run3.details

,datapoint,node_label,trace_idx,comparison,key,actual,expected,distance,node_idx,trace_name,node_name,kwargs,comp_id
0,pikachus,0,2,None,content,Pikachus are dangerous creatures that smoke to...,They are:\n * dangerous\n * smoke cigarettes,0.443801,0,chat,chat,{},0
1,goldens,0,1,chat_eval,.,"[{'id': 'dog1', 'text': 'The Golden Retriever ...","The Golden Retriever is a friendly, intelligen...",0.100000,0,get_docs,get_docs,{'system_prompt': 'how close are these 2 docum...,0
2,goldens,1,2,None,content,"The Golden Retriever is a friendly, intelligen...",They are:\n * friendly\n * intelligent\n,0.647368,1,chat,chat,{},1


In [ ]:
rows = list(run3.details.itertuples())
r= rows[0]
r

Pandas(Index=0, datapoint='pikachus', node_label='0', trace_idx=2, comparison=None, key='content', actual='Pikachus are dangerous creatures that smoke tons of cigarettes and scare children.', expected='They are:\n  * dangerous\n  * smoke cigarettes', distance=0.4438011800616313, node_idx=0, trace_name='chat', node_name='chat', kwargs={}, comp_id=0)

## Comparing Runs

In [ ]:
# TODO pprint
# have a utility function that prints the summary
# and have a utility function that returns the k datapoints that regressed the most
# have a pprint version of it that actually plots the traces and the difference between them



In [ ]:
#| export
import math
from typing import Optional
import textwrap

In [ ]:
#| export
class Comparison(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    from_run: TestSetRun
    to_run: TestSetRun
    log_dir: Optional[Path] = None
    summary: pd.DataFrame
    details: pd.DataFrame

    def __repr__(self):     
        return (
            f"Comparison(\n"
            f"  from_run={textwrap.indent(self.from_run.__repr__(), '  ').strip()}, \n"
            f"  to_run={textwrap.indent(self.to_run.__repr__(), '  ').strip()}, \n"
            f"  summary=Dataframe({self.summary.shape}), \n"
            f"  details=Dataframe({self.details.shape}), \n"
            f")")

    def __str__(self):
        return self.__repr__()

In [ ]:
#| export
def sort_conditions(df):
    return df.sort_values(by=['node_idx','comparison','expected'])

def limit_to_datapoint(df,datapoint):
    return df.loc[df['datapoint'] == datapoint]

def get_datapoint(ds,datapoint):
    return sort_conditions(limit_to_datapoint(ds.details,datapoint))


def describe_changes(ds1,ds2,datapoint,epsilon=1e-3):
    """
    Describe the changes between two datapoints
    """
    
    # get the detailed version of the datasets and limit to only rows of the given datapoint
    datapoint_df1 = get_datapoint(ds1,datapoint)
    datapoint_df2 = get_datapoint(ds2,datapoint)

    # since these datapoints or not extended or reduced, we expect the same set of expected nodes and the same set of tuples of the type (content,comparison)
    # lets assert this in the code
    assert datapoint_df1.shape == datapoint_df2.shape, f"Datapoint {datapoint} has different number of rows in the two datasets {ds1} and {ds2}"

    changes = []

    # get the first comparison whose node aligned to a different trace
    for row1,row2 in zip(datapoint_df1.itertuples(),datapoint_df2.itertuples()):
        if row1.trace_idx != row2.trace_idx:
            changes.append({
                'datapoint':datapoint,
                'change_type':'alignment_change',
                'before':row1.trace_idx,
                'after':row2.trace_idx,
                'comparison_id':row1.comp_id,
            })
            break
            

    for row1,row2 in zip(datapoint_df1.itertuples(),datapoint_df2.itertuples()):
        if math.isclose(row1.distance,row2.distance,abs_tol=epsilon):
            continue
        if row2.distance + epsilon > row1.distance:
            change_types = 'regression'
        elif row2.distance - epsilon < row1.distance:
            change_types = 'improvement'
        else:
            continue
        
        changes.append({
            'datapoint':datapoint,
            'change_type':change_types,
            'value':row1.distance - row2.distance,
            'comparison_id':row1.comp_id,
            'node_label':row1.node_label,
            'expected':row1.expected,
            'before':row1.actual,
            'after':row2.actual,
        })
    
    return pd.DataFrame(changes)


            

In [ ]:
describe_changes(run1,run2,'goldens')

,datapoint,change_type,value,comparison_id,node_label,expected,before,after
0,goldens,improvement,0.048492,1,1,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","Golden Retrievers are friendly, intelligent do..."


In [ ]:
#| export
def compare_datasets(ds1,ds2,epsilon=1e-3,log_dir=None):
    """
    Compare two datasets
    """
    
    summary_1 = ds1.summary.sort_values(by='datapoint')
    summary_2 = ds2.summary.sort_values(by='datapoint')

    changed_datapoints = []
    change_summary = []
    detailed_changes = []

    for row1,row2 in zip(summary_1.itertuples(),summary_2.itertuples()):
        datapoint = row1.datapoint
        distance_change = not math.isclose(row1.distance,row2.distance,abs_tol=epsilon)
        coverage_change = row1.coverage != row2.coverage

        if distance_change or coverage_change:
            changed_datapoints.append(datapoint)

            detailed_change = describe_changes(ds1,ds2,datapoint,epsilon)
            detailed_changes.append(detailed_change)

            change_types = set(detailed_change['change_type'])
            if 'alignment_change' in change_types:
                alignment_change = True
            else:
                alignment_change = False
            
            if 'improvement' in change_types and not 'regression' in change_types:
                score_change = 'improved'
            elif 'regression' in change_types and not 'improvement' in change_types:
                score_change = 'regressed'
            else:
                score_change = 'changed'
            
            total_score_change = row1.distance-row2.distance

            change_summary.append({
                'datapoint':datapoint,
                'alignment_change':alignment_change,
                'score_change_type':score_change,
                'total_score_change':total_score_change,
            })

    changes_summary = pd.DataFrame(change_summary)
    if len(detailed_changes) > 0:
        detailed_changes = pd.concat(detailed_changes)
    else:
        detailed_changes = pd.DataFrame()

    return Comparison(
        from_run=ds1,
        to_run=ds2,
        summary=changes_summary,
        details=detailed_changes,
        log_dir=log_dir,
    )

In [ ]:
comparison_2 = compare_datasets(run1,run2)
comparison_2

Comparison(
  from_run=TestSetRun(
    test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
    log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/trace_logs/v001, 
    summary=Dataframe((2, 5)), 
    details=Dataframe((3, 13)), 
    debug=dict), 
  to_run=TestSetRun(
    test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
    log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/trace_logs/v002, 
    summary=Dataframe((2, 5)), 
    details=Dataframe((3, 13)), 
    debug=dict), 
  summary=Dataframe((2, 4)), 
  details=Dataframe((2, 8)), 
)

In [ ]:
comparison_2.summary

,datapoint,alignment_change,score_change_type,total_score_change
0,goldens,False,improved,0.048492
1,pikachus,False,improved,0.124780


In [ ]:
comparison_2.details

,datapoint,change_type,value,comparison_id,node_label,expected,before,after
0,goldens,improvement,0.048492,1,1,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","Golden Retrievers are friendly, intelligent do..."
0,pikachus,improvement,0.124780,0,0,They are:\n * dangerous\n * smoke cigarettes,"I'm sorry, but I can only provide information ...","Pikachus are not dogs, they are fictional crea..."


In [ ]:
comparison_3 = compare_datasets(run1,run3)
comparison_3.summary


,datapoint,alignment_change,score_change_type,total_score_change
0,goldens,False,improved,0.004006
1,pikachus,False,improved,0.499794


In [ ]:
comparison_3.details

,datapoint,change_type,value,comparison_id,node_label,expected,before,after
0,goldens,improvement,0.004006,1,1,They are:\n * friendly\n * intelligent\n,"The Golden Retriever is a friendly, intelligen...","The Golden Retriever is a friendly, intelligen..."
0,pikachus,improvement,0.499794,0,0,They are:\n * dangerous\n * smoke cigarettes,"I'm sorry, but I can only provide information ...",Pikachus are dangerous creatures that smoke to...


In [ ]:
run1

TestSetRun(
  test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
  log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/trace_logs/v001, 
  summary=Dataframe((2, 5)), 
  details=Dataframe((3, 13)), 
  debug=dict)

In [ ]:
# then continue to make the pprint of a change class with topk changes of each type

In [ ]:
# TODO when pprinting comparisons, give urls to files, so that its easy in nb to navigate to them

## Eval main entrypoint

In [ ]:
# TODO make eval_dataset save the expected yaml files as serialized objects, and use them to be indempotent
# same with comparisons

In [ ]:
#| export
from typing import Callable,Dict,List,Optional,Tuple

In [ ]:
#| export
EVAL_COMPARISONS = {
    'eq':eq,
    'eval':safe_eval,
    'chat_eval':chat_eval,
    'cosine_dist':cosine_dist,
}

EVAL_DEFAULT_COMPARISON = cosine_dist

In [ ]:
# TODO  we make eval dataset load_from output dir
    # we check if its the same up to yaml whitespace, by comparing the yaml string after loading and serializing


In [ ]:
#| export
async def eval(
  test_dir:Path,
  out_dir:Path,
  agents:List[Tuple[str,DiagramSchema]],
  k:Optional[int]=5,
  force_run:bool=False,
  silent:bool=False,
  comparisons: Optional[Dict[str,Callable]]=None,
  default_comparison:Optional[Callable]=None,
  ):
  """
  The main eval function.
  Evaluates a set of agents on a set of tests.
  Compares the results of all agents to the first agent.
  pprints a summary of the results to the console.
  and saves all files to the out_dir.

  Args:
    tests_dir: Path to the directory containing the tests.
    out_dir: Path to the directory to write the results to.
    agents: A list of tuples of agent names and their DiagramSchema.
    k: The number of datapoints to print to the summary at most. Defaults to 5.
    force_run: If True, deletes out dir content and reruns the agents. 
      If False, we skip the agents that have already been run.
      Defaults to False.
    silent: If True, dont pprint comparisons. Defaults to False.
    comparisons: A dictionary of comparison names and their functions, to add to the allowed comparisons.
      Defaults to None.
    default_comparison: The default comparison function to use if no comparison is specified.
      This is used to compare the first agent to the rest of the agents.
      Defaults to stringdale.eval.cosine_dist
  """

  global EVAL_COMPARISONS
  global EVAL_DEFAULT_COMPARISON
  if comparisons is not None:
    comparisons = EVAL_COMPARISONS | comparisons
  else: 
    comparisons = EVAL_COMPARISONS
  if default_comparison is None:
    default_comparison = EVAL_DEFAULT_COMPARISON

  eval_dataset_tasks = []
  for agent_name,agent_schema in agents:
    log_dir = f'{out_dir}/logs/{agent_name}'
    eval_dataset_tasks.append(eval_dataset(
      Agent=agent_schema,
      test_dir=test_dir,
      log_dir=log_dir,
      comparisons=comparisons,
      default_comparison=default_comparison))

  datasets = await asyncio.gather(*eval_dataset_tasks,return_exceptions=True)

  for result,(agent_name,_) in zip(datasets,agents):
    if isinstance(result,Exception):
      result.args = (f"When evaluating agent {agent_name}:\n{result.args[0]}", )+ result.args[1:]
      raise result

  first_dataset = datasets[0]
  comparisons = []
  for dataset in datasets[1:]:
    comparisons.append(compare_datasets(first_dataset,dataset))
  
  return datasets,comparisons

  # then, we pprint the summary of each comparison seperately

  # then we group the per datapoint comps across all dataset comparisons by the datapoint id
  # and for each datapoint we pprint a combined datapoint comparison.
  # combined datapoints for each datapoint, the total metrics of each version
  # and then for each comparison that is different from baseline, say how it is different for every version.

  # we return an EvalResult object that tracks the input of the EvalData, but also has the DataSet and DataSetComp objects for each dataset and comparison
  
  pass


In [ ]:
agents = [
    (version,load_agent(conf_dir/version))
    for version in ['v001','v002','v003']
]
agents

[('v001', <stringdale.base.DiagramSchema>),
 ('v002', <stringdale.base.DiagramSchema>),
 ('v003', <stringdale.base.DiagramSchema>)]

In [ ]:
test_dir = get_git_root() / "sample_data" / "eval_datasets" / "test_cases"
eval_out = get_git_root() / "sample_data" / "eval_datasets" / "eval_out"

In [ ]:
res = await eval(
    test_dir=test_dir,
    out_dir=eval_out,
    agents=agents,
)

In [ ]:
d,c = res

In [ ]:
d

[TestSetRun(
   test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
   log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/eval_out/logs/v001, 
   summary=Dataframe((2, 5)), 
   details=Dataframe((3, 13)), 
   debug=dict),
 TestSetRun(
   test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
   log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/eval_out/logs/v002, 
   summary=Dataframe((2, 5)), 
   details=Dataframe((3, 13)), 
   debug=dict),
 TestSetRun(
   test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
   log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/eval_out/logs/v003, 
   summary=Dataframe((2, 5)), 
   details=Dataframe((3, 13)), 
   debug=dict)]

In [ ]:
c

[Comparison(
   from_run=TestSetRun(
     test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
     log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/eval_out/logs/v001, 
     summary=Dataframe((2, 5)), 
     details=Dataframe((3, 13)), 
     debug=dict), 
   to_run=TestSetRun(
     test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
     log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/eval_out/logs/v002, 
     summary=Dataframe((2, 5)), 
     details=Dataframe((3, 13)), 
     debug=dict), 
   summary=Dataframe((1, 4)), 
   details=Dataframe((1, 8)), 
 ),
 Comparison(
   from_run=TestSetRun(
     test_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/test_cases, 
     log_dir=/Users/dean/dl/stringdale/sample_data/eval_datasets/eval_out/logs/v001, 
     summary=Dataframe((2, 5)), 
     details=Dataframe((3, 13)), 
     debug=dict), 
   to_run=TestSetRun(
     test_dir=/Users/dean/dl/stringdale/sample_data/eval_datase

In [ ]:
# TODO 
# in the end, we want few entrypoints:

# eval, eval_single, and align_trace, validate_tests (just to make sure they are formatted correctly)

# eval will get lists of versions, and which comparisons to do, and log dir to save results to etc..


# TODO add to the tutorial a performance section. Explain the lazy eval of distances
# TODO add to "V" ability to specifiy the funcname for presenting in the drawing


In [ ]:
# explain in tutorial

# how do we do training and validation on workflows?

# we have 2 expected datasets, train and test

# we look at the total distance of the validation set to see that we are improving on it

# but we only look at the comparisons and fix our configs or diagrams based on the training set

In [ ]:
# TODO add asyncio sempathores to Chat and DB operations etc, so that we dont get rate limited due to tons of async requests

""" # Optional semaphore
from contextlib import asynccontextmanager
from typing import Optional

@asynccontextmanager
async def optional_semaphore(semaphore: Optional[asyncio.Semaphore] = None):
    if semaphore is not None:
        async with semaphore:
            yield
    else:
        yield

# Usage example:
async def my_function(limit_concurrency: bool = False):
    sem = asyncio.Semaphore(2) if limit_concurrency else None
    
    async with optional_semaphore(sem):
        # Your async code here
        await asyncio.sleep(1)
        print("Function executed")


"""

## export

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()